In [7]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


--2024-06-19 10:06:47--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-19 10:06:47 (36.9 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [14]:
import minsearch
import json

In [15]:
with open('documents.json', 'rt') as data_file:
    docs_raw = json.load(data_file)

In [20]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [22]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [30]:
index = minsearch.Index(text_fields=['question',"text", "section"], 
               keyword_fields=["course"])

In [31]:
index.fit(documents)

In [32]:
q = 'The course has started, can i still enroll?'

In [40]:
boost ={'question':3.0,'section':0.5}

results = index.search(
    query = q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost, num_results=5
    
)

In [41]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [42]:
from openai import OpenAI

In [43]:
client = OpenAI()

In [44]:
response = client.chat.completions.create(model ='gpt-4o', messages =[{"role":"user", "content":q}])

In [46]:
response.choices[0].message.content

"It's possible to enroll in a course after it has started, but it depends on the specific policies of the institution or the course provider. Here are some general steps you can take:\n\n1. **Check the Enrollment Policy**: Look on the course's official webpage or contact the institution directly to see if late enrollment is allowed.\n \n2. **Contact the Instructor or Administration**: If the policy isn't clear, you can reach out directly to the course instructor or the administrative office to inquire about the possibility of late enrollment.\n\n3. **Review Course Materials**: If allowed to enroll late, see if you can get access to any missed lectures, notes, assignments, or other course materials to catch up.\n\n4. **Understand the Implications**: Be aware that enrolling late might require additional effort to make up missed work and keep up with ongoing assignments.\n\n5. **Formalize Your Enrollment**: If granted permission, ensure that you complete any necessary administrative steps

In [54]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT form the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION:{question}

CONTEXT:
{context}
""".strip()

In [50]:
context = ""

for doc in results:
    context = context + f"section:{doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [52]:
print(context)

section:General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section:General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section:General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with t

In [58]:
prompt=prompt_template.format(question=q, context=context).strip()

In [59]:
prompt

"You're a course teaching assistant. Answer the QUESTION based on the CONTEXT form the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\nIf the CONTEXT doesn't contain the answer, output NONE\n\nQUESTION:The course has started, can i still enroll?\n\nCONTEXT:\nsection:General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection:General course-related questions\nquestion: Course - Can I follow the course after it finishes?\nanswer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on you

In [60]:
response = client.chat.completions.create(model ='gpt-4o', messages =[{"role":"user", "content":prompt}])

In [61]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."